In [45]:
def pcap_parce(file_in, file_out):
    """
    This function is intended to parse capillary pressure data form the Neftecom standard file into csv file loadable into Techlog
    Arguments:
        file_in <string> fine name which needs to be parsed. Needs to be in 'xlsx' format
        file_out <string> name of the output file
    """
    from openpyxl import load_workbook # import lib to process excel files
    wb2 = load_workbook(file_in)

    
    # Create an output file and write into it curves headers and units
    header = 'SampleID; MD; LITH; CPOR; CKKL; CSWA; PClab\n TEXT; m; TEXT; %; mD; %; atm\n'

    with open(file_out, 'w') as file1:
        for line in header:
            file1.write(line)
        
    SampleID = wb2.sheetnames
    MD = []
    LITH = []
    Poro = []
    Perm = []
    CSW = []
    PC = []

    # Process excel file and extract data from each worksheet
    for sheet in wb2:
        top_depth = sheet['O24'].value.split('-')[0]
        top_depth = float(top_depth.replace(',','.'))
        position = sheet['P24'].value
        MD.append(top_depth + position)
        LITH.append(sheet['Q24'].value)
        Poro.append(sheet['S24'].value)
        Perm.append(sheet['R24'].value)
    
        for row in sheet.iter_cols(min_row = 24, min_col = 20, max_row = 25, max_col = 29, values_only = True):
            PC.append(0.9678 * row[0]) # Coefficient at pressure is to convert form kgf|cm2 to atm
            CSW.append(row[1])
    

    file1 = open(file_out, 'a+')
    
    # Write data into output file suitable to load into Techlog as array data
    for i in range(len(SampleID)):
        for j in range(i*10,(i+1)*10):
            row = map(str,[SampleID[i], MD[i], LITH[i], Poro[i], Perm[i], CSW[j], PC[j]])
            fin = ';'.join(row)
            file1.writelines(fin)
            file1.write('\n')          
    file1.close()

file_in = 'ККД_603р_10.xlsx'
file_out = '603R_PCAP.csv'

pcap_parce(file_in, file_out)
